In [1]:
import pprint
import requests     # 2.18.4
import json  
import pandas as pd
import re
import numpy as np
import os

# Recursos

* https://www.pythonsherpa.com/tutorials/2/ --how to connect API
* https://data.metabolismofcities.org/library/maps/577245/view/ shapefile Barris Barcelona
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/sanitat-hospitals-atencio-primaria/resource/9e135848-eb0a-4bc5-8e60-de558213b3ed Hospitals i centres d'atencio primaria
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/punts-informacio-turistica/resource/31431b23-d5b9-42b8-bcd0-a84da9d8c7fa
Sitios de interes turistico
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-sanitat/resource/2bf62aa3-63c8-4177-b57a-e32a8c7eb275 farmacias + hospitals 
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-educacio/resource/29d9ff10-6892-4f16-9012-d5c4997857e7 equipament educació
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/trams-aparcament-superficie/resource/26f6ecdd-b0ac-46c4-84f0-a312640913ca trams aparcament BCN
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/culturailleure-parcsjardins/resource/b64d32a8-aea5-47a8-9826-479b211f5d46 parcs i jardins
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-cultura-i-lleure/resource/f3721b17-bf9e-4bdd-853c-cb6200e1b442 cultural
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/arbrat-parcs/resource/23076aaa-4f0e-4045-b4e5-61d5e651b5a6 arbres parcs
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/arbrat-zona/resource/29cd5c1f-11b1-404b-b3a5-ae29940b8c55 arbres BCN
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/arbrat-viari/resource/23124fd5-521f-40f8-85b8-efb1e71c2ec8 arbres viari
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/np-nasia-miradors/resource/50e946a8-f4a2-4a16-8b40-6553bf4ccd81 
miradors
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-cultura-i-lleure/resource/f3721b17-bf9e-4bdd-853c-cb6200e1b442 equipament cultura i lleure
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-esports/resource/6409e71a-6c79-4d21-9c14-373dbd01f26d equipaments esport
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-animals-i-plantes/resource/0b57a185-8986-4d0f-922f-da8415056575 equipaments animals i plantes
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/educacio-ensenyament-infantil/resource/114a72b3-2c9c-40c6-a7e6-d85dca704726 equipaments educacio infantils (0-6)
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/culturailleure-espaismusicacopes/resource/062da2e7-ddc9-4659-807a-2c1c5918b73c espais de musica, bars, pubs, karaokes...
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/culturailleure-bibliotequesimuseus/resource/d4803f9b-5f01-48d5-aeef-4ebbd76c5fd7 biblioteques i museus
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/culturailleure-cinemesteatresauditoris/resource/0f706441-b9d8-47c9-9e71-ced453810a72 cinemas i teatres 
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/grans-centres-comercials/resource/d9153b84-2694-4c4f-8805-c861d4e98863 grans centres comercials
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/np-piscines/resource/0508ed3c-362b-4f1b-8980-bbdc06358155 psicinas cobertes i descobertes
* https://opendata-ajuntament.barcelona.cat/data/es/dataset/renda-disponible-llars-bcn/resource/34ededd1-6271-4b9b-bac9-18a3fc264a48 Renta disponible per llar BCN 2018 
* https://www.ine.es/jaxiT3/Datos.htm?t=2861 poblacion Barcelona ciutat 2017
* https://ajuntament.barcelona.cat/estadistica/castella/Anuaris/Anuaris/anuari18/cap08/C0803020.htm Total delictes barcelona per districte 2017 (ultim any...)



Falta mirar de ruido, aire (solo hay de Barcelona en agregado...) mas datos, pensar..


In [21]:
# Base url for package information. This is always the same.
base_url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id='

url_API = {
# 'dataset' : '5539d248-2fe0-4ecc-89f5-c695ec7efa3c', # dataset 2022_01_Gener_ITINERARIS_ITINERARIS
# 'terrazas2' : '878e188b-7efa-4695-acb6-b9527d7f2794', # datset terrazas + limit (todos resultados)
# 'hospitales' : '9e135848-eb0a-4bc5-8e60-de558213b3ed', # 
# 'interes_turistico' : '31431b23-d5b9-42b8-bcd0-a84da9d8c7fa',
# 'farmacias' : '2bf62aa3-63c8-4177-b57a-e32a8c7eb275',
# 'educacion' : '29d9ff10-6892-4f16-9012-d5c4997857e7',
# 'aparcamientos' : '26f6ecdd-b0ac-46c4-84f0-a312640913ca',
# 'parques' : 'b64d32a8-aea5-47a8-9826-479b211f5d46',
# 'cultural' : 'f3721b17-bf9e-4bdd-853c-cb6200e1b442',
# 'arbres_parcs' : '23076aaa-4f0e-4045-b4e5-61d5e651b5a6',
# 'arbres_bcn' : '29cd5c1f-11b1-404b-b3a5-ae29940b8c55',
'arbres_viaris' : '23124fd5-521f-40f8-85b8-efb1e71c2ec8',
# 'miradors' : '50e946a8-f4a2-4a16-8b40-6553bf4ccd81',
# 'cultural_lleure' : 'f3721b17-bf9e-4bdd-853c-cb6200e1b442',
# 'equip_esport' : '6409e71a-6c79-4d21-9c14-373dbd01f26d',
# 'equip_animals' : '0b57a185-8986-4d0f-922f-da8415056575',
# 'edud_infantil' : '114a72b3-2c9c-40c6-a7e6-d85dca704726',
# 'bar_copas' : '062da2e7-ddc9-4659-807a-2c1c5918b73c',
# 'biblio_museus' : 'd4803f9b-5f01-48d5-aeef-4ebbd76c5fd7',
# 'cinema_teatres' : '0f706441-b9d8-47c9-9e71-ced453810a72',
# 'centres_comercials' : 'd9153b84-2694-4c4f-8805-c861d4e98863',
# 'psicinas' : '0508ed3c-362b-4f1b-8980-bbdc06358155'
}


# var_names = ['dataset','terrazas2','hospitales','interes_turistico','farmacias','educacion','aparcamientos','parques',
#             'cultural','arbres_parcs','arbres_bcn','arbres_viaris','miradors','cultural_lleure','equip_esport','equip_animals',
#             'edud_infantil','bar_copas','biblio_museus','cinema_teatres','centres_comercials','psicinas']

limit = '&limit=200000'


sql = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search_sql?sql=SELECT * from "f8bcf3ac-dec6-4099-ba4b-eeca9abf4ea3"'

# # Construct the url for the package of interest
# package_information_url = base_url + url_API['arbres_parcs'] + limit

# # Make the HTTP request
# package_information = requests.get(package_information_url)

# # Use the json module to load CKAN's response into a dictionary
# package_dict = json.loads(package_information.content)

# # Check the contents of the response.
# assert package_dict['success'] is True  # again make sure if response is OK
# package_dict = package_dict['result']   # we only need the 'result' part from the dictionary

# pprint.pprint(package_dict)  

In [22]:
def get_data_API():
    for var,url in url_API.items():
        package_information_url = base_url + url + limit

        # Make the HTTP request
        package_information = requests.get(package_information_url)

        # Use the json module to load CKAN's response into a dictionary
        package_dict = json.loads(package_information.content)

        # Check the contents of the response.
        print(var)
        try: 
            assert package_dict['success'] is True  # again make sure if response is OK
            package_dict = package_dict['result']   # we only need the 'result' part from the dictionary

            data_api = pd.DataFrame(package_dict['records'])

            data_api.to_csv('dataset_opendata\\{}.csv'.format(var),encoding='utf-8-sig')
        except:
            AssertionError

# url_API['aparcamientos']
get_data_API()

arbres_viaris


In [4]:
# data_api = pd.DataFrame(package_dict['records'])
# data_api

In [5]:
# data_api.to_csv('dataset_opendata\\aparcamientos.csv',encoding='utf-8-sig')

In [6]:
# os.getcwd()

In [7]:
# package_dict

In [8]:
# data_api.columns

In [9]:
# hospitales.columns
# sorted(hospitales.addresses_zip_code.unique()) # cp de hospitales

In [10]:
# len(data_api.NOM_DISTRICTE.unique()), data_api.NOM_DISTRICTE.unique()

In [11]:
# data_api.NOM_BARRI.unique()

In [12]:
# os.getcwd()

In [13]:
# # Datos Iealista
# data = pd.read_csv('C:/Users/ggari/Desktop/Master_MESIO/TFM/srapper/data_scrapping/complete_csv_22-03-12.csv')
# # Datos hospitales api Open data
# hospitales = pd.read_csv('C:/Users/ggari/Desktop/Master_MESIO/TFM/srapper/dataset_opendata/hospitals_atencio_primaria.csv')

In [14]:
# hospitales.columns